In [11]:
import json
import logging
import os
import os.path as osp
import pickle
import random
from collections import defaultdict, Counter
from copy import deepcopy

import h5py
import numpy as np
import torch
from matplotlib import pyplot as plt
from tqdm import tqdm

from cvpods.data import DATASETS
from cvpods.data.base_dataset import BaseDataset
from cvpods.data.detection_utils import read_image
from cvpods.structures import Boxes, BoxMode
from cvpods.structures.boxes import matched_boxlist_iou
from cvpods.utils.distributed import is_main_process, synchronize, get_rank
from cvpods.data.datasets.bi_lvl_rsmp import resampling_dict_generation, apply_resampling, resampling_dict_generation_ent, \
    apply_resampling_ent
from cvpods.data.datasets.paths_route import _PREDEFINED_SPLITS_VisualPhrase_SGDET
from cvpods.data.datasets.rel_utils import annotations_to_relationship
from cvpods.utils.dump.intermediate_dumper import add_dataset_metadata
import xmltodict

In [5]:
"""
This file contains functions to parse COCO-format annotations into dicts in "cvpods format".
"""

'\nThis file contains functions to parse COCO-format annotations into dicts in "cvpods format".\n'

In [6]:
logger = logging.getLogger("cvpods." + __name__)

In [7]:
!pip install pascal-voc-writer

In [8]:
!pip install xmltodict

  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)


In [9]:


with open("/home/naitik/projects/SGG_Continual/data/visual_phrase/VOC3000/Annotations/3000_000001.xml") as file:
    file_data = file.read() # read file contents
    
    # parse data using package
    dict_data = xmltodict.parse(file_data)

In [10]:
dict_data

{'annotation': {'folder': 'VOC3000',
  'filename': '3000_000001.jpg',
  'source': {'database': 'VOC3000',
   'annotation': 'Amin Sadeghi',
   'image': 'bing'},
  'size': {'width': '800', 'height': '600', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'person_nextto_car',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '1',
     'ymin': '1.060363e+01',
     'xmax': '7.556813e+02',
     'ymax': '5.903964e+02'}},
   {'name': 'person',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '3.594212e+02',
     'ymin': '2.166183e+01',
     'xmax': '6.756037e+02',
     'ymax': '5.967656e+02'}},
   {'name': 'car',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '6.790672e+00',
     'ymin': '1.286443e+01',
     'xmax': '6.939924e+02',
     'ymax': '5.816280e+02'}}]}}

In [17]:
dict_data['annotation'].keys()

dict_keys(['folder', 'filename', 'source', 'size', 'segmented', 'object'])

In [18]:
dict_data['annotation']['filename']

'3000_000001.jpg'

In [ ]:
#@DATASETS.register()

                                                                    
  

In [ ]:
class UIUCVisualPhraseDataset(BaseDataset):
    
    def __init__(self, cfg, dataset_name, transforms=[], is_train=True):
        
        super(VGStanfordDataset, self).__init__(cfg, dataset_name, transforms, is_train)
        
        self.cfg = cfg
        
        
        if 'train' in dataset_name:
            self.split = 'train'
        elif 'val' in dataset_name:
            self.split = 'val'
        elif 'test' in dataset_name:
            self.split = 'test'
        
        
        self.check_img_file = True 
        
        self.name = dataset_name
        
        self.filenames, self.annotations_xml = load_image_filenames(self.data_root , self.check_img_file)
        
    def __len__(self):
        
        assert(len(self.filenames) == len(self.annotations_xml))
        
        return len(self.filenames)
    
    def _load_annotations(self):
        
        pass
    
    def _get_metadata(self): 
        
        
        
    
    
        
                                                                    

In [16]:
import os 

In [ ]:
dir_list = os.listdir(path)

In [92]:
def load_image_filenames(img_dir, check_img_file = True):
    """
    Loads the image filenames of Visual Phrase dataset.
    
    Parameters:
        image_file: JSON file. Elements contain the param "image_id".
        img_dir: directory where the VisualGenome images are located
    Return:
        List of filenames corresponding to the good images
        
    """
    
    file_list_annotations = [img_dir+ "Annotations/" + x for x in os.listdir(img_dir+"Annotations")]
    file_list_images_reorderd = []
    file_names_from_images = os.listdir(img_dir+"JPEGImages")
    
    for image_file in file_list_annotations :
        
        if ".xml" in image_file :
            
            with open(image_file) as file:
                
                file_data = file.read() # read file contents
                
                
                # parse data using package
                dict_data = xmltodict.parse(file_data)['annotation']
                
                if check_img_file : 
                    if  dict_data['filename'] in file_names_from_images:
                        file_list_images_reorderd.append(dict_data['filename'])
                        
    file_list_images_reorderd = [img_dir+ "JPEGImages/" + x for x in file_list_images_reorderd]
    
    assert(len(file_list_images_reorderd) == len(file_names_from_annotations))
    
    return file_list_images_reorderd, file_names_from_annotations

In [19]:
import os

In [26]:
path = "/home/naitik/projects/SGG_Continual/data/visual_phrase/VOC3000/JPEGImages"
dir_list = [path + x for x in os.listdir(path)]

In [88]:
img_dir = "/home/naitik/projects/SGG_Continual/data/visual_phrase/VOC3000/"
check_img_file = True 




In [85]:
len(file_list_images_reorderd)

2978

In [81]:
file_list_images_reorderd[540]

'/home/naitik/projects/SGG_Continual/data/visual_phrase/VOC3000/JPEGImages/3000_002067.jpg'

In [82]:
file_names_from_annotations[540]

'/home/naitik/projects/SGG_Continual/data/visual_phrase/VOC3000/Annotations/3000_002067.xml'

In [96]:
with open(file_names_from_annotations[1]) as file:
    file_data = file.read() # read file contents
    
    # parse data using package
    dict_data = xmltodict.parse(file_data)

dict_data['annotation']

{'folder': 'VOC3000',
 'filename': '3000_002148.jpg',
 'source': {'database': 'VOC3000',
  'annotation': 'Amin Sadeghi',
  'image': 'bing'},
 'size': {'width': '600', 'height': '713', 'depth': '3'},
 'segmented': '0',
 'object': [{'name': 'person_running',
   'pose': 'Unspecified',
   'truncated': '1',
   'difficult': '0',
   'bndbox': {'xmin': '1',
    'ymin': '1.065933e+01',
    'xmax': '1.998303e+02',
    'ymax': '5.943588e+02'}},
  {'name': 'person_running',
   'pose': 'Unspecified',
   'truncated': '1',
   'difficult': '0',
   'bndbox': {'xmin': '1.074728e+02',
    'ymin': '1.423575e+00',
    'xmax': '3.808510e+02',
    'ymax': '6.682448e+02'}},
  {'name': 'person_running',
   'pose': 'Unspecified',
   'truncated': '1',
   'difficult': '0',
   'bndbox': {'xmin': '3.531438e+02',
    'ymin': '1.250648e+01',
    'xmax': '5.470946e+02',
    'ymax': '6.756334e+02'}},
  {'name': 'person_running',
   'pose': 'Unspecified',
   'truncated': '1',
   'difficult': '0',
   'bndbox': {'xmin': '

In [42]:
len(file_list)

2978

In [32]:
file_list = [img_dir+ "Annotations/" + x for x in os.listdir(img_dir+"Annotations")]

In [34]:
len(file_list)

2979

In [ ]:
self_try = file_names_from_annotations

def _get_metadata(self_try):
    
    obj_to_idx = {'aeroplane' : 0 , 'bicycle' : 1 , 'bird' : 2 ,  'bottle' : 3 ,  'car' : 4 ,  'cat' : 5 ,  'chair' : 6 ,  'dog' : 7,  'horse' : 8,  'person' : 9,  'sofa' : 10}
    rel_to_idx = {'flying_in_sky' : 0 , 'parked_at_airport' : 1 , 'nextto' : 2 , 'lying_on' : 3 , 'jumping' : 4 , 'running' : 5 , 'drinking' : 6 , 'jumping_on' : 7 , 'lying_in_beach' : 8 , 'riding' : 9 , 'sitting_on' : 10}
    
    idx_to_obj = {}
    
    for key , data in obj_to_idx.items():
        idx_to_obj[data] = key
    
    idx_to_rel = {}
    
    for key , data in rel_to_idx.items():
        idx_to_rel[data] = key
    
    unique_relation_list = []
    unique_attribute_list = []
    
    
    for xml in range(len(self_try)):
        
        with open(xml) as file:
            
            file_data = file.read() # read file contents
            
            # parse data using package
            dict_data = xmltodict.parse(file_data)['annotation']    